In [1]:
!pip install CFGraph --upgrade  -q

In [2]:
from rdflib import URIRef, Graph
from CFGraph import CFGraph
from pprint import pprint
ppl = lambda o: pprint(list(o), width=160)
pps = lambda o: pprint(set(o), width=160)

s1 = URIRef("http://example.org/s1")
s2 = URIRef("http://example.org/s2")
s3 = URIRef("http://example.org/s3")
s4 = URIRef("http://example.org/s4")

p = URIRef("http://a.example/p1")
o = URIRef("http://example.org/o1")


**Our graph has for subjects:**

* `<"http://example.org/s1">`: an RDF collection with 8 elements - seven literals with "b" appearing twice and a URI.
* `<"http://example.org/s2">`: an RDF list with 7 elements -- six literals ("b" is a duplicate) and a URI.  This list is unordered
* `<"http://example.org/s3">`: an empty RDF collection.  This returns `rdf:nil` as a normal graph and an empty list in CGGraph
* `<"http://example.org/s4">`: an RDF collection with a single URI element

In [3]:
rdf = f"""
<{s1}> <{p}> ("a" "b" "c" "d" "e" <{o}> "b").
<{s2}> <{p}> "a", "b", "c", "d", "e", <{o}>, "b".
<{s3}> <{p}> ().
<{s4}> <{p}> (<{o}>).
"""

print(rdf)


<http://example.org/s1> <http://a.example/p1> ("a" "b" "c" "d" "e" <http://example.org/o1> "b").
<http://example.org/s2> <http://a.example/p1> "a", "b", "c", "d", "e", <http://example.org/o1>, "b".
<http://example.org/s3> <http://a.example/p1> ().
<http://example.org/s4> <http://a.example/p1> (<http://example.org/o1>).



In [4]:
# Load a Collection Flattening Graph
cg = CFGraph()
_ = cg.parse(data=rdf, format="turtle")

# Load a "vanilla" RDF graph for comparison
g = Graph()
_ = g.parse(data=rdf, format="turtle")

**The CGGraph returns all of the elements in the collection for the first predicate as a list.**


In [5]:
ppl(cg.triples((s1, p, None)))

[(rdflib.term.URIRef('http://example.org/s1'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('a')),
 (rdflib.term.URIRef('http://example.org/s1'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('b')),
 (rdflib.term.URIRef('http://example.org/s1'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('c')),
 (rdflib.term.URIRef('http://example.org/s1'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('d')),
 (rdflib.term.URIRef('http://example.org/s1'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('e')),
 (rdflib.term.URIRef('http://example.org/s1'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.URIRef('http://example.org/o1')),
 (rdflib.term.URIRef('http://example.org/s1'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('b'))]


**The Graph returns the first node**

In [6]:
ppl(g.triples((s1, p, None)))

[(rdflib.term.URIRef('http://example.org/s1'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.BNode('fbd3921aea969422cbddfb80667bfe930b1'))]


**The output of the CGGraph and Graph are the same with lists**

In [7]:
pps(cg.triples((s2, p, None)))
print("Graphs are equal = " + str(set(cg.triples((s2, p, None))) == set(g.triples((s2, p, None)))))

{(rdflib.term.URIRef('http://example.org/s2'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.URIRef('http://example.org/o1')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('a')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('b')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('c')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('d')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.Literal('e'))}
Graphs are equal = True


**The empty list returns a null list in CGGraph and `rdf:nil` in a plain Graph**

In [8]:
ppl(cg.triples((s3, p, None)))
ppl(g.triples((s3, p, None)))

[]
[(rdflib.term.URIRef('http://example.org/s3'), rdflib.term.URIRef('http://a.example/p1'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#nil'))]


**All the other *forward* graph functions behave as expected**

In [9]:
ppl(cg.predicate_objects(s4))
ppl(cg.objects(s4, p))

[(rdflib.term.URIRef('http://a.example/p1'), rdflib.term.BNode('feda7e2d44fd640b6ad14065e44f4f559b8'))]
[rdflib.term.URIRef('http://example.org/o1')]


***Reverse* functions are not implemented in this version**

In [10]:
print("*** subject_predicates ***")
ppl(cg.subject_predicates(o))
print("\n*** subjects ***")
ppl(cg.subjects(p, o))
print("\n*** subject_objects ***")
ppl(cg.subject_objects(p))

*** subject_predicates ***
[(rdflib.term.BNode('feda7e2d44fd640b6ad14065e44f4f559b6'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first')),
 (rdflib.term.BNode('feda7e2d44fd640b6ad14065e44f4f559b8'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.URIRef('http://a.example/p1'))]

*** subjects ***
[rdflib.term.URIRef('http://example.org/s2')]

*** subject_objects ***
[(rdflib.term.URIRef('http://example.org/s2'), rdflib.term.Literal('a')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.Literal('b')),
 (rdflib.term.URIRef('http://example.org/s4'), rdflib.term.BNode('feda7e2d44fd640b6ad14065e44f4f559b8')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.Literal('c')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.URIRef('http://example.org/o1')),
 (rdflib.term.URIRef('http://example.org/s2'), rdflib.term.Literal('e')),
 (rdflib.term.URIRef('http://exa